In [ ]:
from IPython.core.display import HTML
import requests
import time
import pandas as pd
import datetime
HTML("""
<style>
.container { width:100% !important; }
</style>
""")

In [ ]:
 api_key ='H89X3Q1Y9FUJEJ99'

In [ ]:
def _get_data(symbols,time_from,time_to,api_key):
    url = f"https://www.alphavantage.co/query?function=NEWS_SENTIMENT&tickers={symbols}&time_from={time_from}&time_to={time_to}&limit=1000&apikey={api_key}"
    r = requests.get(url)
    data = r.json()
    return data

def _get_label_sentiment(x):
    if x <= -0.35:
        return 'Bearish','Bearish'
    elif -0.35 < x <= -0.15:
        return 'Somewhat-Bearish','Bearish'
    elif -0.15 < x < 0.15:
        return 'Neutral','Neutral'
    elif 0.15 <= x < 0.35:
        return 'Somewhat_Bullish','Bullish'
    else:  # x >= 0.35
        return 'Bullish','Bullish'

In [ ]:

def get_dataset(time_from="20230410T0130",
                time_to='',
                MAX_API_CALLS_PER_DAY = 25,
                MAX_API_CALLS_PER_MIN = 5
               ):
    data_list=[]
    for i in range(1,MAX_API_CALLS_PER_DAY+1):
        if i%5==0:
            time.sleep(60)

        data=_get_data('TSLA',time_from,time_to,api_key)
        if 'feed' not in data:break
        if len(data['feed'])==0: break
        data_list.append(data)
        time_to=data['feed'][-1]['time_published'][:-2] # Take all the way up to last 2 since api only takes minute level granularity
    df=pd.concat([pd.DataFrame(data['feed']) for data in data_list])
    # Extract TSLA specific relevance (we didn't use it in video)
    df['ticker_relevance_TSLA']=df['ticker_sentiment'].apply(lambda l:[el for el in l if el['ticker']=='TSLA'][0]['relevance_score']).astype(float)
    # Extract TSLA specific sentiment
    df['ticker_sentiment_TSLA']=df['ticker_sentiment'].apply(lambda l:[el for el in l if el['ticker']=='TSLA'][0]['ticker_sentiment_score']).astype(float)
    # Only take tickers with TSLA in headline
    df=df[df.title.str.contains('tsla|tesla',case=False)]
    # Extract # of tickers
    df['num_tickers']=df.ticker_sentiment.apply(lambda l:len(l))
    # Only take when # of tickers = 1
    df = df[df.num_tickers==1]
    # Applying the function and creating two new columns
    df[['detailed_original_label','label']] = df.apply(lambda row: _get_label_sentiment(row['ticker_sentiment_TSLA']), axis=1, result_type='expand')
    # Drop duplicates..
    df.drop_duplicates(subset=['summary'],inplace=True,keep='first')

    df.set_index('time_published',inplace=True)

    df.sort_index(inplace=True)
    return df



In [ ]:
df = get_dataset(time_to='')
df.to_csv('tsla_sentiment.csv')
df

In [ ]:
df = df.drop(df.columns[[1,2,3,4,5,6,7,8, 9,11,12,13,14,15]], axis=1)

In [ ]:
import pandas as pd


dataset_path = '/content/drive/MyDrive/dhoni/tsla_sentiment.csv'
df = pd.read_csv(dataset_path)

In [ ]:

df['time_published'] = pd.to_datetime(df['time_published'], format='%Y%m%dT%H%M%S').dt.strftime('%Y-%m-%d %H:%M:%S+00:00')


print(df)


                 time_published  overall_sentiment_score
0     2023-12-20 01:39:42+00:00                 0.014493
1     2023-12-20 03:27:00+00:00                 0.335030
2     2023-12-20 08:06:00+00:00                 0.221843
3     2023-12-20 08:13:45+00:00                 0.193866
4     2023-12-20 10:41:21+00:00                 0.254833
...                         ...                      ...
1748  2024-09-13 13:00:16+00:00                 0.223789
1749  2024-09-15 01:41:09+00:00                 0.111687
1750  2024-09-16 02:37:22+00:00                -0.159767
1751  2024-09-16 14:26:13+00:00                 0.000000
1752  2024-09-16 15:00:25+00:00                 0.156448

[1753 rows x 2 columns]


In [ ]:
df['time_published'] = pd.to_datetime(df['time_published'])
df['time_published'] = df['time_published'].dt.date
df

,time_published,overall_sentiment_score
0,2023-12-20,0.014493
1,2023-12-20,0.335030
2,2023-12-20,0.221843
3,2023-12-20,0.193866
4,2023-12-20,0.254833
...,...,...
1748,2024-09-13,0.223789
1749,2024-09-15,0.111687
1750,2024-09-16,-0.159767
1751,2024-09-16,0.000000


In [ ]:
from google.colab import drive
import pandas as pd
drive.mount('/content/drive')
file_path = '/content/drive/MyDrive/dhoni/dataset1.csv'

df.to_csv(file_path, index=False)
